In [1]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
from sklearn.model_selection import ParameterGrid
from prophet import Prophet
from tqdm.notebook import tqdm
import plotly.express as px
from sklearn.metrics import root_mean_squared_error, root_mean_squared_log_error
import plotly.graph_objects as go 

from prophet.diagnostics import cross_validation 
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
import warnings
warnings.filterwarnings("ignore")

In [2]:
def plot_predictions(train, test):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=train["ds"][-30:],
            y=train["y"][-30:],
            name="Train",
            line=dict(color="#2A61EB"),
        )
    )

    test_connected = pd.concat([train.tail(1), test])

    fig.add_trace(
        go.Scatter(
            x=test_connected["ds"], y=test_connected["y"], name="Test", line=dict(color="#2456D2")
        )
    )

    fig.add_trace(
        go.Scatter(
            x=test_connected["ds"],
            y=test_connected["yhat"],
            name="Forecast",
            line=dict(color="#159A20", dash = 'dash'),
        )
    )

    return fig 

In [3]:
df = pd.read_parquet("train.parquet")
df = df.drop(columns=["city", "state", "type", "transactions"])
df = df[[col for col in df.columns if not col.startswith("holiday")]]
df = df.sort_values(by=["store_nbr", "family", "date"])

df["dcoilwtico_interpolate"] = df["dcoilwtico_interpolate"].bfill()
df = df.rename(columns={"dcoilwtico_interpolate": "oil", "sales": "y", "date": "ds"})

In [4]:
periods = 16

In [5]:
holiday_df = pd.read_csv("holidays_events.csv")
holiday_df = holiday_df[["date", "description"]]
holiday_df = holiday_df.rename(columns={"date": "ds", "description": "holiday"})
holiday_df["lower_window"] = -1
holiday_df["upper_window"] = 1

In [9]:
x = df[["store_nbr", "family"]].drop_duplicates().sample(1)
store_nbr = x["store_nbr"].iloc[0]

family = x["family"].iloc[0]

temp = df[(df["store_nbr"] == store_nbr) & (df["family"] == family)]
px.line(temp, x="ds", y="y", title=f"{family} - {store_nbr}")

In [10]:
df_sample = df[(df["store_nbr"] == store_nbr) & (df["family"] == family)].sort_values('ds')
train, test = df_sample.iloc[:-periods], df_sample.iloc[-periods:]

In [11]:
m = Prophet(
    seasonality_mode="additive",
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    holidays=holiday_df,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10,
    holidays_prior_scale=10,
)
m.add_regressor("onpromotion")
m.add_regressor("oil")
m.add_regressor("month")
m.add_regressor("day_of_month")
m.add_regressor("day_of_year")
m.add_regressor("week_of_month")
m.add_regressor("day_of_week")
m.add_regressor("year")
m.add_regressor("is_wknd")
m.add_regressor("quarter")
m.add_regressor("is_month_start")
m.add_regressor("is_month_end")
m.add_regressor("is_quarter_start")
m.add_regressor("is_quarter_end")
m.add_regressor("is_year_start")
m.add_regressor("is_year_end")

m.fit(train)

00:42:20 - cmdstanpy - INFO - Chain [1] start processing
00:42:20 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
future = m.make_future_dataframe(periods=periods, include_history=False)
future = future.merge(
    test[
        [
            "ds",
            "onpromotion",
            "oil",
            "month",
            "day_of_month",
            "day_of_year",
            "week_of_month",
            "day_of_week",
            "year",
            "is_wknd",
            "quarter",
            "is_month_start",
            "is_month_end",
            "is_quarter_start",
            "is_quarter_end",
            "is_year_start",
            "is_year_end",
        ]
    ], 
    on='ds', 
    how='left'
)


forecast = m.predict(future)[['ds','yhat']]
forecast['yhat'] = forecast['yhat'].clip(lower=0)

test = test[['ds','y']].merge(forecast, on = ['ds'])

In [13]:
plot_predictions(train, test)

In [14]:
rmsle = round(root_mean_squared_log_error(test['y'], test['yhat']),2) 
rmse = round(root_mean_squared_error(test['y'], test['yhat']),2)

print(f"RMSE: {rmse}, RMSLE: {rmsle}")

RMSE: 185.87, RMSLE: 0.41


In [15]:
df_cv = cross_validation(model=m, period='30 days', initial='1200 days', horizon='16 days', parallel='processes')

In [16]:
performance_metrics(df_cv).iloc[:,1:].mean().T.round(2)

mse         5557.71
rmse          73.92
mae           54.96
mdape          0.12
smape          0.15
coverage       0.65
dtype: float64

In [17]:
param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.05, 0.1, 0.5],  # Added default 0.05
    "seasonality_prior_scale": [
        0.01,
        0.1,
        0.5,
        1.0,
        10.0,
        12.0,
        15.0,
    ],  # Added 0.5 for finer tuning
    "seasonality_mode": ["additive", "multiplicative"],
    "holidays_prior_scale": [0.01, 0.1, 1.0, 10.0],
    "growth": ["linear", "logistic"],
}

parameters = list(ParameterGrid(param_grid))

In [18]:
df_sample = df[(df["store_nbr"] == store_nbr) & (df["family"] == family)].sort_values('ds')
train, test = df_sample.iloc[:-periods], df_sample.iloc[-periods:]

In [19]:
def run_train(train, test, params):
    m = Prophet(
        seasonality_mode=params["seasonality_mode"],
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        holidays=holiday_df,
        changepoint_prior_scale=params["changepoint_prior_scale"],
        seasonality_prior_scale=params["seasonality_prior_scale"],
        holidays_prior_scale=params["holidays_prior_scale"],
    )
    m.add_regressor("onpromotion")
    m.add_regressor("oil")
    m.add_regressor("month")
    m.add_regressor("day_of_month")
    m.add_regressor("day_of_year")
    m.add_regressor("week_of_month")
    m.add_regressor("day_of_week")
    m.add_regressor("year")
    m.add_regressor("is_wknd")
    m.add_regressor("quarter")
    m.add_regressor("is_month_start")
    m.add_regressor("is_month_end")
    m.add_regressor("is_quarter_start")
    m.add_regressor("is_quarter_end")
    m.add_regressor("is_year_start")
    m.add_regressor("is_year_end")

    m.fit(train)

    future = m.make_future_dataframe(periods=periods, include_history=False)
    future = future.merge(
        test[
            [
                "ds",
                "onpromotion",
                "oil",
                "month",
                "day_of_month",
                "day_of_year",
                "week_of_month",
                "day_of_week",
                "year",
                "is_wknd",
                "quarter",
                "is_month_start",
                "is_month_end",
                "is_quarter_start",
                "is_quarter_end",
                "is_year_start",
                "is_year_end",
            ]
        ],
        on="ds",
        how="left",
    )

    forecast = m.predict(future)[["ds", "yhat"]]
    forecast["yhat"] = forecast["yhat"].clip(lower=0)

    test_score = test[["ds", "y"]].merge(forecast, on=["ds"])

    rmsle = round(root_mean_squared_log_error(test_score["y"], test_score["yhat"]), 2)
    rmse = round(root_mean_squared_error(test_score["y"], test_score["yhat"]), 2)

    df_cv = cross_validation(
        model=m,
        period="30 days",
        initial="1200 days",
        horizon="16 days",
        parallel="threads",
    )

    performance = performance_metrics(df_cv).iloc[:, 1:].mean().T.round(2).to_dict()


 
    return {
            'rmse': rmse, 
            'rmsle': rmsle, 
            'cross_validation_result': performance
        }
    

In [20]:
from joblib import Parallel, delayed 

In [23]:
jobs = []

for params in tqdm(parameters[:40]):
    jobs.append(delayed(run_train)(train, test, params))

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
p = Parallel(n_jobs=12, verbose=10, batch_size=32)(jobs) 

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


In [ ]:
tuning_results = []
for params in tqdm(parameters[:10]):

    m , rmse, rmsle  = run_train(train, test, params)

    df_cv = cross_validation(
        model=m,
        period="30 days",
        initial="1200 days",
        horizon="16 days",
        parallel="threads",
    )

    performance = performance_metrics(df_cv).iloc[:, 1:].mean().T.round(2).to_dict()


    tuning_results.append(
        {
            'rmse': rmse, 
            'rmsle': rmsle, 
            'cross_validation_result': performance
        }
    )

  0%|          | 0/3 [00:00<?, ?it/s]

00:31:55 - cmdstanpy - INFO - Chain [1] start processing
00:31:55 - cmdstanpy - INFO - Chain [1] done processing
00:32:03 - cmdstanpy - INFO - Chain [1] start processing
00:32:03 - cmdstanpy - INFO - Chain [1] start processing
00:32:03 - cmdstanpy - INFO - Chain [1] done processing
00:32:03 - cmdstanpy - INFO - Chain [1] done processing
00:32:03 - cmdstanpy - INFO - Chain [1] start processing
00:32:04 - cmdstanpy - INFO - Chain [1] done processing
00:32:05 - cmdstanpy - INFO - Chain [1] start processing
00:32:05 - cmdstanpy - INFO - Chain [1] done processing
00:32:05 - cmdstanpy - INFO - Chain [1] start processing
00:32:05 - cmdstanpy - INFO - Chain [1] start processing
00:32:05 - cmdstanpy - INFO - Chain [1] start processing
00:32:05 - cmdstanpy - INFO - Chain [1] start processing
00:32:06 - cmdstanpy - INFO - Chain [1] done processing
00:32:06 - cmdstanpy - INFO - Chain [1] done processing
00:32:06 - cmdstanpy - INFO - Chain [1] start processing
00:32:06 - cmdstanpy - INFO - Chain [1